# Lets Grow More

# Task 1: Advanced

# Develop A Neural Network That Can Read Handwriting

# Importing the Libraries  and load the dataset

In [41]:
import keras
from keras.datasets import mnist

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()


# Data Preprocessing

In [42]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)


# Convert class vectors to binary class matrices (one-hot encoding)

In [43]:
num_classes = 10  # There are 10 digits (0 to 9)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Printing the shape

In [44]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Creating the Model

In [45]:
batch_size = 128
num_classes = 10
epochs = 10
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

# Train the model

In [50]:
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")
model.save('mnist.h8')
print("Saving the model as mnist.h8")

Epoch 1/10
469/469 [==============================] - 71s 149ms/step - loss: 0.8967 - accuracy: 0.7442 - val_loss: 0.6861 - val_accuracy: 0.8483
Epoch 2/10
469/469 [==============================] - 73s 156ms/step - loss: 0.8322 - accuracy: 0.7605 - val_loss: 0.6280 - val_accuracy: 0.8567
Epoch 3/10
469/469 [==============================] - 73s 157ms/step - loss: 0.7795 - accuracy: 0.7711 - val_loss: 0.5807 - val_accuracy: 0.8654
Epoch 4/10
469/469 [==============================] - 76s 163ms/step - loss: 0.7318 - accuracy: 0.7843 - val_loss: 0.5414 - val_accuracy: 0.8700
Epoch 5/10
469/469 [==============================] - 75s 160ms/step - loss: 0.6946 - accuracy: 0.7942 - val_loss: 0.5092 - val_accuracy: 0.8775
Epoch 6/10
469/469 [==============================] - 82s 175ms/step - loss: 0.6632 - accuracy: 0.8023 - val_loss: 0.4824 - val_accuracy: 0.8805
Epoch 7/10
469/469 [==============================] - 87s 185ms/step - loss: 0.6361 - accuracy: 0.8077 - val_loss: 0.4604 - val_ac

INFO:tensorflow:Assets written to: mnist.h8\assets


Saving the model as mnist.h8


# Evaluate the model

In [51]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.4123612344264984
Test accuracy: 0.8919000029563904


# Creating a GUI

In [52]:
from keras.models import load_model
from tkinter import *
import tkinter as tk

import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [60]:

model = load_model('mnist.h8')
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
app = App()
mainloop()

1/1 [==============================] - 0s 21ms/step
